In [889]:
import pandas as pd
import numpy as np

In [890]:
train = pd.read_csv("./input/train.csv")
test = pd.read_csv("./input/test.csv")

In [891]:
print("train null")
print(train.isna().sum())
print("")
print("-----------")
print("")
print("test null")
print(test.isna().sum())

train null
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

-----------

test null
PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64


In [892]:
test["Fare"] = test["Fare"].fillna(test["Fare"].median())
train["Embarked"] = train["Embarked"].fillna("S")

train = train.drop(columns=["Ticket"], axis=1)
test = test.drop(columns=["Ticket"], axis=1)

In [893]:
def setCabin(val):
    result = 0
    if pd.isna(val): result = 0
    else: result = 1
    return result

In [894]:
train["HasCabin"] = train["Cabin"].apply(setCabin)
test["HasCabin"] = test["Cabin"].apply(setCabin)

In [895]:
train["Cabin"] = train["Cabin"].fillna("N")
test["Cabin"] = test["Cabin"].fillna("N")

In [896]:
train["Cabin"] = train["Cabin"].str[:1]
test["Cabin"] = test["Cabin"].str[:1]

In [897]:
train = pd.get_dummies(train, columns=["Sex", "Embarked"], drop_first=True)
test = pd.get_dummies(test, columns=["Sex", "Embarked"], drop_first=True)

In [898]:
def setPclass(val):
    return (4 - val)
    
train["Pclass"] = train["Pclass"].apply(setPclass)
test["Pclass"] = test["Pclass"].apply(setPclass)

In [899]:
train["FareBand"] = pd.cut(train["Fare"], 4)
train[["FareBand", "Survived"]].groupby(['FareBand'], as_index=False).mean().sort_values(by='FareBand', ascending=True)

,FareBand,Survived
0,"(-0.512, 128.082]",0.368113
1,"(128.082, 256.165]",0.724138
2,"(256.165, 384.247]",0.666667
3,"(384.247, 512.329]",1.000000


In [900]:
def setFare(val):
    result = 0
    if val < 170.766: result = 0
    elif val < 341.553: result = 1
    else: result = 2
    return result

train["Fare"] = train["Fare"].apply(setFare)
test["Fare"] = test["Fare"].apply(setFare)

train = train.drop(columns=["FareBand"], axis=1)

In [901]:
# train[["Cabin", "Survived"]].groupby(['Cabin'], as_index=False).count()

In [902]:
train["FamilySize"] = train["SibSp"] + train["Parch"]
test["FamilySize"] = test["SibSp"] + test["Parch"]
train = train.drop(columns=["SibSp", "Parch"], axis=1)
test = test.drop(columns=["SibSp", "Parch"], axis=1)

In [903]:
def setAlone(val):
    if val == 0 : return 1
    else: return 0

train["IsAlone"] = train["FamilySize"].apply(setAlone)
test["IsAlone"] = test["FamilySize"].apply(setAlone)

In [904]:
train["FamilyBand"] = pd.cut(train["FamilySize"], 3)
train[["FamilyBand", "Survived"]].groupby(['FamilyBand'], as_index=False).mean().sort_values(by='FamilyBand', ascending=True)

,FamilyBand,Survived
0,"(-0.01, 3.333]",0.400483
1,"(3.333, 6.667]",0.204082
2,"(6.667, 10.0]",0.000000


In [905]:
def setFamily(val):
    result = 0
    if val < 3.333: result = 2
    elif val < 6.667: result = 1
    else: result = 0
    return result

train["FamilySize"] = train["FamilySize"].apply(setFamily)
test["FamilySize"] = test["FamilySize"].apply(setFamily)

train = train.drop(columns=["FamilyBand"], axis=1)

In [906]:
train.head()

,PassengerId,Survived,Pclass,Name,Age,Fare,Cabin,HasCabin,Sex_male,Embarked_Q,Embarked_S,FamilySize,IsAlone
0,1,0,1,"Braund, Mr. Owen Harris",22.0,0,N,0,1,0,1,2,0
1,2,1,3,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,0,C,1,0,0,0,2,0
2,3,1,1,"Heikkinen, Miss. Laina",26.0,0,N,0,0,0,1,2,1
3,4,1,3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,0,C,1,0,0,1,2,0
4,5,0,1,"Allen, Mr. William Henry",35.0,0,N,0,1,0,1,2,1


In [907]:
train["Title"] = train["Name"].str.extract(', ([A-Z][a-z]+)')
test["Title"] = test["Name"].str.extract(', ([A-Z][a-z]+)')

In [908]:
def setTitle(val):
    result = 0
    if val == "Mr": result = 0
    elif val == "Miss": result = 2
    elif val == "Mrs": result = 3
    else: result = 1
    return result

train["Title"] = train["Title"].apply(setTitle)
test["Title"] = test["Title"].apply(setTitle)

In [909]:
train = train.drop(columns=["Name"], axis=1)
test = test.drop(columns=["Name"], axis=1)

In [910]:
def setCabin2(val):
    result = 0
    if val == "A": result = 6
    elif val == "B": result = 1
    elif val == "C": result = 2
    elif val == "D": result = 3
    elif val == "E": result = 4
    elif val == "F": result = 5
    elif val == "G": result = 0
    elif val == "T": result = 7    
    else: result = 8
    return result

In [911]:
train["Cabin"] = train["Cabin"].apply(setCabin2)
test["Cabin"] = test["Cabin"].apply(setCabin2)

In [912]:
# train["CabinBand"] = pd.cut(train["Cabin"], 9)
# train[["CabinBand", "Survived"]].groupby(['CabinBand'], as_index=False).mean().sort_values(by='CabinBand', ascending=True)

In [913]:
train.head()

,PassengerId,Survived,Pclass,Age,Fare,Cabin,HasCabin,Sex_male,Embarked_Q,Embarked_S,FamilySize,IsAlone,Title
0,1,0,1,22.0,0,8,0,1,0,1,2,0,0
1,2,1,3,38.0,0,2,1,0,0,0,2,0,3
2,3,1,1,26.0,0,8,0,0,0,1,2,1,2
3,4,1,3,35.0,0,2,1,0,0,1,2,0,3
4,5,0,1,35.0,0,8,0,1,0,1,2,1,0


In [914]:
train_age_yes = train[train["Age"].notna()]
train_age_no = train[train["Age"].isna()]

In [915]:
x = train_age_yes[["HasCabin", "IsAlone"]]
y = train_age_yes["Age"]

In [916]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, make_scorer
score = make_scorer(mean_squared_error)
k_fold = KFold(n_splits=200, shuffle=True, random_state=42)

In [917]:
lr = LinearRegression()
lr.fit(x, y)
scoring = "mean_squared_error"
score = cross_val_score(lr, x, y, cv=k_fold, n_jobs=1, scoring=score)
print("LinearRegression: {0: .4f}".format(round(np.mean(np.sqrt(score)))))

LinearRegression:  13.0000


In [918]:
y_predict = lr.predict(train_age_no[["HasCabin", "IsAlone"]])
train_age_no["Age"] = y_predict
train = pd.concat([train_age_yes, train_age_no])

/Users/skyepodium/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [919]:
test_age_yes = test[test["Age"].notna()]
test_age_no = test[test["Age"].isna()]

In [920]:
x_test = test_age_no[["HasCabin", "IsAlone"]]
y_predict = lr.predict(x_test)
test_age_no["Age"] = y_predict
test = pd.concat([test_age_yes, test_age_no])

/Users/skyepodium/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [921]:
train["AgeBand"] = pd.cut(train["Age"], 3)
train[["AgeBand", "Survived"]].groupby(['AgeBand'], as_index=False).mean().sort_values(by='AgeBand', ascending=True)

,AgeBand,Survived
0,"(0.34, 26.947]",0.400560
1,"(26.947, 53.473]",0.378099
2,"(53.473, 80.0]",0.320000


In [922]:
def setAge(val):
    if val < 26.947: result = 2
    elif val < 53.473: result = 1
    else: result = 0
    return result

In [923]:
train["Age"] = train["Age"].apply(setAge)
test["Age"] = test["Age"].apply(setAge)
train = train.drop(columns=["AgeBand"], axis=1)

In [924]:
train.head()

,PassengerId,Survived,Pclass,Age,Fare,Cabin,HasCabin,Sex_male,Embarked_Q,Embarked_S,FamilySize,IsAlone,Title
0,1,0,1,2,0,8,0,1,0,1,2,0,0
1,2,1,3,1,0,2,1,0,0,0,2,0,3
2,3,1,1,2,0,8,0,0,0,1,2,1,2
3,4,1,3,1,0,2,1,0,0,1,2,0,3
4,5,0,1,1,0,8,0,1,0,1,2,1,0


In [925]:
x = train.drop(columns=["PassengerId", "Survived"], axis=1)
y = train["Survived"]

In [926]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
import lightgbm as lgb

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
# k_fold = KFold(n_splits=200, shuffle=True, random_state=42)
k_fold = KFold(n_splits=200, shuffle=True, random_state=42)
scoring = 'accuracy'

In [927]:
xgb_model = xgb.XGBClassifier(objective="binary:logistic", random_state=42)
xgb_model.fit(x, y)
scoring = 'accuracy'
score = cross_val_score(xgb_model, x, y, cv=k_fold, n_jobs=1, scoring=scoring)
print("xgboost: {0: .4f}".format(round(np.mean(score)*100, 2)))

xgboost:  81.8500


In [928]:
lgb_model = lgb.LGBMClassifier(objective="binary", random_state=42)
lgb_model.fit(x, y)
scoring = 'accuracy'
score = cross_val_score(lgb_model, x, y, cv=k_fold, n_jobs=1, scoring=scoring)
print("lightgbm: {0: .4f}".format(round(np.mean(score)*100, 2)))

lightgbm:  83.9500


In [929]:
x_test = test.drop(columns=["PassengerId"], axis=1)

In [930]:
predict = lgb_model.predict(x_test)

In [931]:
submission = pd.DataFrame({
    "PassengerId": test["PassengerId"],
    "Survived": predict
})

In [933]:
submission.to_csv("0328_onehot_lgb_model_{0: .4f}.csv".format(83.95), index=False)